# WELCOME TO OUR WORKING LABORATORY

This is a place where we train and export model and test new algorithm of combining boxes in our website

## First let us look at the codes used for training, loading and exporting the model

In [1]:
# Import YOLO from ultralytics for finetuning
from ultralytics import YOLO
# os is used for passing directory path easily
import os
# pickle is used for exporting the model
import pickle

In [2]:
# Loading the pre-trained yolov8m model
model = YOLO("yolov8m.pt")

In [ ]:
# Pass only the path to data.yaml file and this contains path to train and validation splits
results = model.train(data = os.path.join("Dataset", "Dataset_6", "data.yaml"), epochs = 450)

In [ ]:
# For validation of the model
model.val()

In [5]:
# Loading the model directly from weights
model = YOLO(os.path.join("runs", "detect", "train29", "weights", "best.pt"))

In [ ]:
# The following is used if we want to resume the training of the model
model.train(resume=True)

In [11]:
# Pickle dump or exports the model in the .pkl format
pickle.dump(model, open("OW_2.pkl", "wb"))

In [2]:
# The following code is used to load the model with pickle format
model = pickle.load(open("OW_2.pkl", "rb"))

In [ ]:
# printing the neural layers of model
model

# Now we will how the model return the result and how can use them

In [ ]:
# model.predict is used to predict the detection.
result = model.predict(source=os.path.join("Dataset_4", "valid", "images", "14-__page-0001_jpg.rf.1c47d3f0ddd6a0ddcb5cc349f1852a4b.jpg"))

In [14]:
# result[0].boxes.xyxy   # box with xyxy format, (N, 4)
# result[0].boxes.xywh   # box with xywh format, (N, 4)
# result[0].boxes.xyxyn  # box with xyxy format but normalized, (N, 4)
# result[0].boxes.xywhn  # box with xywh format but normalized, (N, 4)
result[0].boxes.conf   # confidence score, (N, 1)
# result[0].boxes.cls    # cls, (N, 1)

tensor([0.6766, 0.6617, 0.6278, 0.6254, 0.6128, 0.6069, 0.6065, 0.5376, 0.5250, 0.5235, 0.5082, 0.5000, 0.4830, 0.4732, 0.4650, 0.4568, 0.4470, 0.4281, 0.4092, 0.4081, 0.3960, 0.3951, 0.3876, 0.3825, 0.3750, 0.3389, 0.3335, 0.3323, 0.3317, 0.3194, 0.3084, 0.2956, 0.2952, 0.2927, 0.2720, 0.2584, 0.2507],
       device='cuda:0')

In [122]:
# importing opencv to read the images and creating bounding boxes
import cv2
# import matplotlib for showing graphs and images
from matplotlib import pyplot as plt
# Image is used to show and store the finalized images.
from PIL import Image

for r in result:
    #im_array = r.plot()  # plot a BGR numpy array of predictions # Use this line only when directly showing the results from YOLO
    im_array = img # Use this line when you have to show your tampered image
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    #im.save('results.jpg')  # save image

In [120]:
# Using opencv to read the image
img = cv2.imread(os.path.join("Dataset_4", "valid", "images", "14-__page-0001_jpg.rf.1c47d3f0ddd6a0ddcb5cc349f1852a4b.jpg"))

In [ ]:
# for converting arrays to numpy
import numpy as np

boxes = result[0].boxes.cpu().numpy() # storing the coordinates of boxes
# print(first)
qu = {} # queue for traversing through coordinates
final = [] # the final list we will obtain
small = [] # saves coordinates of a continuous word
conf = [] # saves the confidance score according to the values in final list
visited = [0] * len(boxes) # keeping a track of visited boxes
for i, box in enumerate(boxes):
    if(box.conf[0] < 0.47):
        continue
    if(visited[i]):
        continue
    visited[i] = 1
    r = box.xyxy[0]
    prev = r
    qu[i] = r
    print(r)
    small = []
    con = box.conf[0]
    small.append(r)
    while qu:
        index, val = list(qu.items())[0]
        del qu[index]
        visited[index] = 1
        prev = val
        print(val)
        for j, b in enumerate(boxes):
            # print(j)
            if(b.conf[0] < 0.47): # HYPERPARAMETER for changing the minimum confidance score required
                continue
            
            if(index == j):
                continue
            if(visited[j]):
                continue

            if( con < b.conf[0]):
                con = b.conf[0]

            p = b.xyxy[0]
            dist = np.sqrt(np.square(p[0] - prev[0]) + np.square(p[1] - prev[1]))
            print(dist)
            if(dist < 9): # HYPERPARAMETER for changing the pixels threshold betweek two boxes
                qu[j] = p
                small.append(p)
                # print(p)
    
        print(small)
        print(con)
    if(con):
        conf.append(round(con, 2))
    if(len(small)):
        final.append(small)
    

In [ ]:
# The following code stores the finalized boxe coordinates
coord = []
for lst in final:
    min_x, min_y, max_x, max_y = 10000.0, 10000.0, -1.0, -1.0
    for lt in lst:
        if(lt[0] < min_x):
            min_x = lt[0]
            min_y = lt[1]
        if(lt[2] > max_x):
            max_x = lt[2]
            max_y = lt[3]
    coord.append([min_x, min_y, max_x, max_y])
print(coord)

In [46]:
# Removing duplicates if there are any
dup = []
for i, lst in enumerate(final):
    if(i == 0):
        continue
    if(final[i-1] == final[i]):
        dup.append(i)

In [ ]:
# Creating bounding box and printing the confidence score over them
for i, box in enumerate(coord):
    r = [v.astype(int) for v in box]
    print(r)                                               # print boxes
    cv2.rectangle(img, r[:2], r[2:], (0, 0, 255), 1)
    #img = cv2.rectangle(img, (r[0], r[1] - 5), (r[2], r[1]), (0, 0, 255), -1)
    text = str(round(conf[i] * 100, 2))
    print(text)
    img = cv2.putText(img, text, (r[0], r[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.23, (0, 0, 255), 1)

In [126]:
# These line are required if we want to print confidence score in percentage
text = str(0.68 * 100) + "%"
text

'68.0%'